In [1]:
import tensorflow as tf

In [4]:
numDimensions = 200
maxSeqLength = 65
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

In [3]:
! ls

idsMatrix.npy	    RNN_200_subb_not_imp.ipynb		  wordList-200.npy
models		    tensorboard				  wordVectors-200.npy
output_rnn.ipynb    test_clean_withuser.csv
RNN_100_subb.ipynb  train_clean_subb_withuser_sorted.csv


In [5]:
import numpy as np
wordsList = np.load('wordList-200.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors-200.npy')

In [6]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [7]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-90000


In [8]:
import pandas as pd
import numpy as np

csv = 'test_clean_withuser.csv'
df = pd.read_csv(csv)
df.head()

,id,text
0,15923,democrats support antifa muslim brotherhood ms...
1,60133,unity demo to oppose the far right in enough i...
2,83681,what the fuck did he do this time
3,65507,user do you get the feeling he is kissing user...
4,12588,user nigga ware da hits at


In [9]:
len(df.text)

240

In [10]:
messages_test = df.text

In [11]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [12]:
def test_id(l,u) :
  numFiles=24
  ids_test = np.zeros((numFiles, maxSeqLength), dtype='int32')
  messageCounter = 0
  for i in range(l,u):
    if messageCounter == 24:
      break
    indexCounter = 0
    cleanedLine = cleanSentences(str(messages_test[i]))
    split = cleanedLine.split()
    for word in split:
      try:
        ids_test[messageCounter][indexCounter] = wordsList.index(word)
      except ValueError:
        ids_test[messageCounter][indexCounter] = 399999
      indexCounter = indexCounter + 1
      if indexCounter >= maxSeqLength:
        break
    messageCounter = messageCounter + 1
  return (ids_test)

In [13]:
def res_gen(predsent):
  res = []
  for item in predsent :
    if item[0] > item[1] :
      res.append(0)
    else :
      res.append(1)
  return(res) 

In [14]:
num = len(df.text)
itr = num // batchSize
res = []
for i in range(itr):
  l = i*24
  u = l+24
  ids = test_id(l,u)
  predictedSentiment = sess.run(prediction, {input_data: ids})
  res_itr = res_gen(predictedSentiment)
  res = res + res_itr
  
  

In [18]:
l

216

In [19]:
u

240

In [20]:
len(res)

240

In [21]:
finalres = res[:]

In [25]:
df.head()

,id,text,output
0,15923,democrats support antifa muslim brotherhood ms...,0
1,60133,unity demo to oppose the far right in enough i...,0
2,83681,what the fuck did he do this time,1
3,65507,user do you get the feeling he is kissing user...,0
4,12588,user nigga ware da hits at,0


In [23]:
df['output'] = finalres

In [24]:
df.output.value_counts()

0    208
1     32
Name: output, dtype: int64

In [26]:
out_df = pd.DataFrame({'id':df.id, 'sentiment':df.output})
out_df.head()

,id,sentiment
0,15923,0
1,60133,0
2,83681,1
3,65507,0
4,12588,0


In [27]:
z = {0 : 'UNT',1 :'TIN'}
out_df['sentiment'] = out_df.sentiment.map(z)

In [28]:
out_df.head()

,id,sentiment
0,15923,UNT
1,60133,UNT
2,83681,TIN
3,65507,UNT
4,12588,UNT


In [29]:
out_df.to_csv('output_rnn_subb.csv',header=None ,index=False ,encoding='utf-8')